In [2]:
import requests
from bs4 import BeautifulSoup
from web_scraping import web_table_to_dataframe
import pandas as pd


In [41]:
response = requests.get(f"https://www.baseball-reference.com/leagues/AL/2022-standard-batting.shtml")
soup = BeautifulSoup(response.content, "html.parser")

In [43]:
web_table_to_dataframe(soup("table")[0])
    

,Tm,#Bat,BatAge,R/G,G,PA,AB,R,H,2B,...,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,LOB
0,Baltimore Orioles,58,27.0,4.16,162,6049,5429,674,1281,275,...,.390,.695,97,2119,95,83,12,43,10,1095
1,Boston Red Sox,54,28.8,4.54,162,6144,5539,735,1427,352,...,.409,.731,102,2268,131,63,12,50,23,1133
2,Chicago White Sox,44,29.3,4.23,162,6123,5611,686,1435,272,...,.387,.698,97,2172,127,73,16,35,9,1117
3,Cleveland Guardians,50,25.9,4.31,162,6163,5558,698,1410,273,...,.383,.699,102,2126,115,81,22,52,36,1156
4,Detroit Tigers,53,27.9,3.44,162,5870,5378,557,1240,235,...,.346,.632,84,1859,108,58,10,44,8,1015
5,Houston Astros,45,29.3,4.55,162,6054,5409,737,1341,284,...,.424,.743,111,2293,118,60,9,42,18,1068
6,Kansas City Royals,55,27.1,3.95,162,6010,5437,640,1327,247,...,.380,.686,93,2064,101,48,20,44,7,1091
7,Los Angeles Angels,66,27.9,3.85,162,5977,5423,623,1265,219,...,.390,.687,94,2116,95,54,25,25,28,1050
8,Minnesota Twins,61,26.9,4.30,162,6113,5476,696,1356,269,...,.401,.718,107,2195,133,62,10,46,11,1126
9,New York Yankees,54,30.2,4.98,162,6172,5422,807,1308,225,...,.426,.751,113,2311,121,70,14,41,36,1093


In [1]:
### extract all mlb data
from dotenv import load_dotenv
load_dotenv(override=True)
import os
from os.path import join
import sys
import time
import pandas as pd

from web_scraping import get_all_mlb_managers, get_all_mlb_teams, get_mlb_season_team_batting_stats, get_mlb_season_team_pitching_stats, get_mlb_season_standings, get_mlb_draft_player_bio


df = get_all_mlb_managers()
df.to_csv(join(os.environ["SportsDB"], "baseball", f"mlb_all_managers.csv"), index=False)

time.sleep(3)

df = get_all_mlb_teams()
df.to_csv(join(os.environ["SportsDB"], "baseball", f"mlb_all_teams.csv"), index=False)

time.sleep(3)

for year in range(2000,2023):
    df = get_mlb_season_team_batting_stats(year)
    df.to_csv(join(os.environ["SportsDB"], "baseball", f"mlb_season_team_batting_stats_{year}.csv"), index=False)
    
    time.sleep(3)
    
    df = get_mlb_season_team_pitching_stats(year)
    df.to_csv(join(os.environ["SportsDB"], "baseball", f"mlb_season_team_pitching_stats_{year}.csv"), index=False)
    
    time.sleep(3)
    
    df = get_mlb_season_standings(year)
    df.to_csv(join(os.environ["SportsDB"], "baseball", f"mlb_season_standings_{year}.csv"), index=False)
    
    time.sleep(3)
    
    df = get_mlb_draft_player_bio(year)
    df.to_csv(join(os.environ["SportsDB"], "baseball", f"mlb_season_draft_player_bios_{year}.csv"), index=False)
    
    time.sleep(3)

# Extract all column headers

In [2]:
import pandas as pd
import sys
from os.path import join
import os
import json
from glob import glob
from dotenv import load_dotenv
load_dotenv(override=True)


metadata = {}

tables = ["all_managers", "all_teams", "team_batting_stats", "team_pitching_stats", "season_standings", 
          "draft_player_bios"]

for table in tables:
    column_names = []
    for found_table in glob(join(os.environ["SportsDB"], "baseball", f"*{table}*.csv")):
        column_names.extend(list(pd.read_csv(found_table).columns))

    column_names = list(set(column_names))
    metadata[table] = {}
    for column_name in column_names:
        metadata[table][column_name] = None

if os.path.isfile("metadata.json"):
    print("metadata.json exists already!!! ")
else:
    with open("metadata.json", "w") as f:
        json.dump(metadata, f)